In [1]:
import numpy as np
import pandas as pd
import os, sys

import json

import urllib.parse
from urllib.parse import unquote
from urllib.parse import urlparse

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

### Чистка URL

In [3]:
# Загружаем файл
file_path = '/data/share/project01/gender_age_dataset.txt'
file_limit = None
rawData = pd.read_csv(file_path, sep='\t', nrows=file_limit  )
rawData.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [4]:
#Процедура. Фильтрует домен из url
def toDomain( url ):
    if url.startswith('http://http') : url = url[7:]
    if url.startswith('http://&referrer=') : url = url[17:]
        
    parsed_url = urlparse( urllib.parse.unquote( url ).strip() )
    if parsed_url.scheme not in ['http','https']: return None

    url = parsed_url.netloc.strip()

    if url.startswith('www.') : url = url[4:]

    dpoint = url.rfind(':')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.find('&')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.rfind('@')     
    if dpoint != -1 : url = url[dpoint+1:]    
       
    return url if url.rfind('.') != -1 else None

#Процедура разбирает JSON и возвращет домен и timestamp
def workupDomain( szDomain ):
    return [[toDomain (value['url'] ), value['timestamp']]  for value in json.loads( szDomain )['visits']]

#Перебираем элементы, сохраняя из данных тока домен и timestamp
rawData['domain'] = rawData['user_json'].apply( workupDomain )
rawData.drop(['user_json'], axis=1, inplace=True)
rawData.head()

,gender,age,uid,domain
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"[[zebra-zoya.ru, 1419688144068], [news.yandex...."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"[[sweetrading.ru, 1419717886224], [sweetrading..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"[[ru.oriflame.com, 1418840296062], [ru.oriflam..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"[[translate-tattoo.ru, 1418217864467], [nadiet..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"[[mail.rambler.ru, 1427272415001], [news.rambl..."


In [ ]:
rawData.to_csv('~/project/users.csv', sep=',', index=False)

### Построение таблицы использования доменов

In [ ]:
#Глобальные листы для формирования столбцов
global gender
gender = []
global age
age = []
global url 
domain = []
global footprint
footprint = []

#Процедура. Пробераем по всем элементам json и формируем из них отдельные строки (для каждого листа)
def extractDomain(row):
    for rowDomain in row['domain'] :
        gender.append( row['gender'] )
        age.append( row['age'] )
        domain.append( rowDomain[0] )
        footprint.append( rowDomain[1] )

#Вызываем обработку каждой строки.Рузультат не сохраняем, он в глобальных списках
rawData.apply( extractDomain, axis = 1 )
#Строем DataFrame для доменов
theFootprints = pd.DataFrame ({'gender' : gender,'age' : age, 'domain' : domain, 'footprint': footprint }) 
theFootprints.info()

In [ ]:
theFootprints.to_csv('~/project/footprints.csv', sep=',', index=False)

### Получаем таблицу весов доменов

In [ ]:
#Отсекаем неопределенные посещения
theFootprints = theFootprints[(theFootprints['age'] != '-') & (theFootprints['gender'] != '-')] 

#Получаем список полей упомянутый в исходном файле
ageList = theFootprints['age'].unique()
genderList = theFootprints['gender'].unique()  #Строго зашили, что их два!
indexList = theFootprints['domain'].unique()
#Получаем список наших признаков
columnList = list(genderList[0] + ageList) + list(genderList[1] + ageList)

#Группируем признаки по домену, возрасту и полу, но индексируем только по домену 
theFootprints = theFootprints.groupby(['domain', 'gender', 'age']).count()
theFootprints.reset_index( inplace=True )
theFootprints.set_index(['domain'], inplace=True)
theFootprints.head()

In [ ]:
#Процедура. вставляем в каждую ячейку пустой таблицы вес этого домена для каждого признака
def fillDomain( row ) :
    resultList = pd.Series(index= columnList)

    szDomain = row[0]
    if szDomain is None or len(szDomain) == 0 or szDomain != szDomain : return resultList 
    
    localDomain = theFootprints.loc[ szDomain ]
    if  isinstance(localDomain, pd.DataFrame) :
        for _, theLine in localDomain.iterrows():
            resultList.at[theLine['gender'] + theLine['age'] ] = theLine['footprint']
    else :
        resultList.at[localDomain['gender'] + localDomain['age'] ] = localDomain['footprint']
        
    return resultList

#Создаем таблицу признаков. Заполняем ее значениями доменов (оптимизация для ускорения)
mainDomain = pd.DataFrame (columns = columnList, index = indexList) 
mainDomain = mainDomain.apply(lambda row: list(row.index) , axis=0)

#Пробегаемся по всем столбцам, заменяя домены на их вес из таблицы групперовки
mainDomain = mainDomain.apply(fillDomain, axis=1)
mainDomain.reset_index( inplace=True )
mainDomain.rename(columns={"index": "domain"}, inplace=True)
mainDomain.head()

In [ ]:
mainDomain.to_csv('~/project/domain.csv', sep=',', index=False)

### Другой подход: собираем Bag Of Words


In [ ]:
# Загружаем файл
#file_limit = None
#theUserCorpus = pd.read_csv('~/project/users.csv', sep=',', nrows=file_limit  )
#theUserCorpus.domain = theUserCorpus.domain.tolist()
#theUserCorpus.head()

In [ ]:
#Процедура, превращающая список доменов в текст для корпуса
def toPlainText(row):
    theCollection = list()
    for rowDomain in row['domain'] :
        if rowDomain[0] is None or len(rowDomain[0]) == 0: continue

        theCollection.append( str(rowDomain[0]) )
        theCollection.append( ';' )

    row['domain'] = str('').join(theCollection).replace(' ', '').replace('-', '').replace('.', '').replace(';', ' ')
    return  row  


#Превращаем домены в текст, uid  прячем в индекс
theUserCorpus = rawData.copy()
theUserCorpus = theUserCorpus.apply( toPlainText, axis = 1 )
theUserCorpus['target'] = theUserCorpus.gender+theUserCorpus.age
theUserCorpus.drop(['gender', 'age'], axis=1, inplace=True )
theUserCorpus.set_index(['uid'], inplace=True)

#Создаем таблицу групп признаков: где чего лежит
uList = theUserCorpus.target.unique()
theTargetName = pd.DataFrame( {'code':range(1, len(uList)+1) }, index = uList )
theTargetName.code.loc['--'] = 0
theTargetName.sort_values('code', inplace=True)

#Генерируем номера групп 
theUserCorpus['targetID'] = theUserCorpus['target'].apply( lambda x:  theTargetName.code.loc[x] )
theUserCorpus.drop(['target'], axis=1, inplace=True )
theUserCorpus.sort_values(by=['targetID'], inplace=True)

#Рассчитываем положения их смещения в общем массиве
theTargetName['len'] = theTargetName['code'].apply( lambda type:  len(theUserCorpus[theUserCorpus.targetID == type]) )
theTargetName['begin'] = [theTargetName[theTargetName.code < type ]['len'].sum() \
                        if type > 0 else 0 \
                        for type in range(0, 11) ]
theTargetName['end'] = [theTargetName[theTargetName.code <= type ]['len'].sum() \
                        if type > 0 else int(theTargetName[theTargetName.code == type ]['len']) \
                        for type in range(0, 11) ]

theTargetName

In [ ]:
#Примерный вид корпуса
theUserCorpus[23340:23346]

In [ ]:
#Процедура токенизации
def tokenise( text ):
    words = [word.lower() for word in word_tokenize(text)]
    return words
#Обучаем векторизатор и генерируем Bag of Words
print ('Size of theUserCorpus is ', len(theUserCorpus) )

theCorpus = list(theUserCorpus['domain'])
print ('Size of theCorpus is ', len(theCorpus) )

theVectorizer = CountVectorizer(tokenizer=tokenise)
theBagOfWords = theVectorizer.fit_transform(theCorpus).toarray()
print('Size of theBagOfWords is ', theBagOfWords.shape)

In [ ]:
#Считаем косинусную БЛИЗОСТЬ
from sklearn.metrics.pairwise import cosine_similarity

#Можно перебрать все индексы, но выборочный десяток показывает, что это довольно бессмысленно
id = 23509 

print( '№', theUserCorpus.index[id], '\tдля типа', 'средняя вероятность', 'ближайший вектор ', 'со значением' )
for type in range( 1, 11) : 
    params = theTargetName[theTargetName.code == type] 
    cos_m = cosine_similarity(theBagOfWords[id].reshape(1,-1), theBagOfWords[params.begin[0] : params.end[0]])

    print( '\t\t\t\t\t', params.index[0], cos_m[0].mean(), '\t', cos_m.argmax(), '\t', cos_m[0][ cos_m.argmax() ] )

### Пытаемся обработать вектора из таблицы весов доменов

In [2]:
# Загружаем файл
file_limit = None
theDomain = pd.read_csv('~/project/domain.csv', sep=',', nrows=file_limit  )
theDomain.head()

,domain,F18-24,F25-34,F>=55,F45-54,F35-44,M18-24,M25-34,M>=55,M45-54,M35-44
0,zebra-zoya.ru,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,news.yandex.ru,174.0,577.0,169.0,229.0,674.0,293.0,1351.0,192.0,550.0,849.0
2,sotovik.ru,4.0,15.0,NaN,1.0,NaN,NaN,36.0,NaN,6.0,6.0
3,sweetrading.ru,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN
4,101.ru,69.0,334.0,26.0,21.0,141.0,45.0,547.0,24.0,27.0,181.0


In [150]:
rawData.loc[1000]

gender                                                    F
age                                                   45-54
uid                    bb25e0b4-95ad-45d6-8177-cd715c8ef0c5
domain    [[2015godkozy.com, 1419844778641], [u768.uvled...
Name: 1000, dtype: object

In [156]:
#Выбираем пользователя
uid = 'bb25e0b4-95ad-45d6-8177-cd715c8ef0c5'

print(rawData[rawData['uid'] == uid][['uid', 'gender', 'age']])

#Получаем его домены
theDomens = [list(domain)[0] for domain in rawData[rawData['uid'] == uid].domain]

#Получаем признаки пользователя
theWard = list(pd.DataFrame(list(rawData[rawData['uid'] == uid].domain)[0] )[0])
theWard = pd.DataFrame(theWard, columns = ['domain'])
theWard['ward'] = 0
theWard = theWard.groupby(['domain']).count()
theWard.reset_index( inplace=True )
theWard

                                       uid gender    age
1000  bb25e0b4-95ad-45d6-8177-cd715c8ef0c5      F  45-54


,domain,ward
0,2015godkozy.com,1
1,cache.betweendigital.com,1
2,cl.lcads.ru,1
3,gismeteo.ru,1
4,go.mail.ru,1
5,links.readme.ru,3
6,memori.qip.ru,9
7,readme.ru,3
8,sobesednik.ru,4
9,u768.uvleda.ru,1


In [157]:
#Добавляем вектор нейзвестного пользователя к вектору известных возростов и полов
theVector = pd.merge(left=theDomain, right=theWard, left_on='domain', right_on='domain')
theVector.set_index(['domain'], inplace=True)
theVector.fillna(0, inplace=True)
for column in theVector.columns:
    mean = theVector[column].mean();
    std = theVector[column].std();
    theVector[column] = (theVector[column]-mean)/std
theVector

,F18-24,F25-34,F>=55,F45-54,F35-44,M18-24,M25-34,M>=55,M45-54,M35-44,ward
domain,,,,,,,,,,,
cache.betweendigital.com,0.187506,1.478390,-0.001951,0.079643,0.227877,0.083184,0.459169,-0.126765,0.083511,0.829110,-0.525456
go.mail.ru,3.122901,2.651076,3.127904,3.140839,3.114004,3.132718,2.982926,3.089832,3.064996,2.931748,-0.525456
sobesednik.ru,-0.224479,-0.423588,-0.068295,-0.209260,-0.281823,-0.316100,-0.385669,-0.334504,-0.133011,-0.257806,0.735639
wow-impulse.ru,-0.315591,-0.348389,-0.130736,-0.212178,-0.278103,-0.228757,-0.086456,0.084324,0.075846,-0.104024,-0.525456
gismeteo.ru,-0.052158,-0.182382,-0.068295,-0.139223,-0.026973,0.005823,0.366276,0.228401,0.186982,-0.022387,-0.525456
2015godkozy.com,-0.329456,-0.352645,-0.271228,-0.305560,-0.305076,-0.333569,-0.397403,-0.438373,-0.439590,-0.462849,-0.525456
memori.qip.ru,-0.400761,-0.470410,-0.435136,-0.388729,-0.404597,-0.385975,-0.500074,-0.441724,-0.481745,-0.502719,2.837465
readme.ru,-0.392838,-0.466863,-0.411720,-0.340579,-0.399017,-0.373497,-0.435538,-0.260790,-0.414681,-0.392603,0.315274
u768.uvleda.ru,-0.400761,-0.471829,-0.435136,-0.407697,-0.412038,-0.395957,-0.501052,-0.451776,-0.485577,-0.504617,-0.525456


In [158]:
#Считаем косинусную БЛИЗОСТЬ
from sklearn.metrics.pairwise import cosine_similarity

cos_m = cosine_similarity(theVector.T.values[10:11], theVector.T.values[:10])
#Тоже как-то не очень
cos_m[0]

array([-0.20891102, -0.26412044, -0.20558462, -0.19980058, -0.21799207,
       -0.21304898, -0.26875615, -0.22785442, -0.23133483, -0.25182198])